In [11]:
import numpy as np
import pandas as pd
import time

path = "C:/Users/Aleksi/ownCloud/Vaasan Yliopisto/Gradu/Raw Data/Firm Data/New/"

In [12]:
def na_value_analyzer(df):
    """This function calculates the sum and % of Na values in each column. Returns DataFrame"""
    
    x = pd.DataFrame(columns=['Column Name','NA count','% NA','non NA'])
    
    for col in df:
        na_count = df[col].isna().sum()
        x = x.append({'Column Name':col, 
                     'NA count':na_count, 
                     '% NA':round((na_count/len(df))*100,2),
                     'non NA':len(df)-na_count}, ignore_index=True)
    return x

### Import data

In [13]:
# BANKRUPT 

# Bankrupt, Active (rescue plan), Active (insolvency proceedings), Dissolved (Bankruptcy)


start_time = time.time()
bankrupt = pd.read_csv(filepath_or_buffer=path+"Bankrupt.txt",sep='\t', encoding='UTF-16', na_values='')

print("Import succesful. Time to execute was: ", round(time.time()-start_time,3),"seconds",
      "Original bankrupt DataFrame size: ", bankrupt.shape, sep='\n')

bankrupt.drop('Unnamed: 0', inplace=True, axis=1)

# drop n.a. values
bankrupt.dropna(inplace=True) # drop NA values rowwise

# indicates bankruptcy
bankrupt['status'] = 1 

print("Bankrupt cleaned df size: ",bankrupt.shape)


Import succesful. Time to execute was: 
0.037
seconds
Original bankrupt DataFrame size: 
(6138, 16)
Bankrupt cleaned df size:  (2621, 16)


In [14]:
# ACTIVE

start_time = time.time()
active = pd.DataFrame() # empty DataFrame

# open txt.- files
for i in range(1,11):
    
    df = pd.read_csv(filepath_or_buffer=path+str(i)+'.txt', sep='\t', encoding='UTF-16', na_values='')
    active = active.append(df, ignore_index=True)

print(1*'\n') 
print("Original  active firm df size: ", active.shape,'Time to execute was: ',
      round(time.time()-start_time,3), sep='\n')

# drop unnecessary values
drop_cols = ['Unnamed: 0','Loans EUR Last avail. yr','Provisions EUR Last avail. yr']
active.drop(drop_cols,axis=1, inplace=True)
active.dropna(inplace=True) 

active['status'] = 0 # indicates non-bankrupt


print("Cleaned active df size: ", active.shape)



Original  active firm df size: 
(500000, 18)
Time to execute was: 
3.321
Cleaned active df size:  (95127, 16)


### Create single DataFrame (df)

In [15]:
# MERGE DATAFRAMES TOGETHER AND REMOVE DUPLICATE ROWS

df = active.append(bankrupt, ignore_index=True)

# change dtype from float to int
df['NACE Rev. 2, core code (4 digits)'] = df['NACE Rev. 2, core code (4 digits)'].astype(int) 
df['Last avail. year'] = df['Last avail. year'].astype(int) # change to int

# company name as index
df = df.set_index('Company name') 

In [16]:
df.columns

Index(['Country ISO code', 'NACE Rev. 2, core code (4 digits)',
       'Last avail. year', 'Operating revenue (Turnover) EUR Last avail. yr',
       'Shareholders funds EUR Last avail. yr',
       'Other shareholders funds EUR Last avail. yr',
       'Capital EUR Last avail. yr', 'Working capital EUR Last avail. yr',
       'Operating P/L [=EBIT] EUR Last avail. yr',
       'P/L for period [=Net income] EUR Last avail. yr',
       'Non-current liabilities EUR Last avail. yr',
       'Current liabilities EUR Last avail. yr',
       'Intangible fixed assets EUR Last avail. yr',
       'Total assets EUR Last avail. yr', 'status'],
      dtype='object')

### Outliers & clean

In [17]:
print('Initial size:',df.shape)

# DUPLICATES ? WHY ? --> remove
duplicate_names = df[df.index.duplicated()].index.tolist() # make a list of duplicate company names
df.drop(duplicate_names, axis=0, inplace=True) # remove all duplicates 
print('DUPLICATES removed:',df.shape)


# Remove with NEGATIVE REVENUE
neg_revenue = df['Operating revenue (Turnover) EUR Last avail. yr']<0
df = df[~neg_revenue]
print('Negative REVENUE removed:',df.shape)


# Remove with "BALANCE SHEET TOTAL" <= 0: "Total shareh. funds & liab." == "Taseen loppusumma"
df = df[df['Total assets EUR Last avail. yr']>0]
print('Zero and negative "BALANCE SHEET TOTAL" removed:',df.shape)


# TOTAL LIABILITIES

# create tot_liab = Current liabilities + Non-current liabilities

df['tot_liab'] = df['Current liabilities EUR Last avail. yr'] + df['Non-current liabilities EUR Last avail. yr']


# Negative total liabilities should be asset? --> remove all negative values
df = df[df['tot_liab']>=0]
print('Negative TOTAL LIABILITIES removed:',df.shape)


# Dividing by 0 is not possible in X4 so values of 0.0 --> 1.0

t = (df['tot_liab']==0).sum()

df.loc[df['tot_liab']==0,'tot_liab'] = 1            # if tot_liab == 0     --> replace with 1

print('"tot_liab" with value of "0.0" replaced with "1.0"',t,'instances')


Initial size: (97748, 15)
DUPLICATES removed: (97738, 15)
Negative REVENUE removed: (97725, 15)
Zero and negative "BALANCE SHEET TOTAL" removed: (97721, 15)
Negative TOTAL LIABILITIES removed: (97658, 16)
"tot_liab" with value of "0.0" replaced with "1.0" 45 instances


In [18]:
# Winsorizing technique not used in thesis!
# df[fin_st_cols] = df[fin_st_cols].clip(lower=df.quantile(0.01), upper=df.quantile(0.99), axis=1)

In [19]:
# SAVE THE FILE LOCALLY INTO .txt
df.to_csv('df.txt')